This program is to get the risk allocation based on correlation between strategies

In [25]:
#Source: https://www.mql5.com/en/articles/16500?utm_source=mql5.com.tg&utm_medium=message&utm_campaign=articles.codes.repost

import pandas as pd
# Replace 'your_file.xlsx' with the path to your file
input_file = 'DBG-SP.xlsx'
# Load the Excel file and skip the first {skiprows} rows
data = pd.read_excel(input_file, skiprows=4518)
# Select the 'profit' column (assumed to be 'Unnamed: 10') and filter rows as per your instructions
data = data[['Time','Profit','Balance']][1:-1] 
data = data[data.index % 2 == 0]  # Filter for rows with odd indices
data[['Profit', 'Balance']] = data[['Profit', 'Balance']].apply(pd.to_numeric, errors='coerce').fillna(0)
data['Time'] = pd.to_datetime(data['Time'], errors='coerce')
profit_loss = data['Profit']
account_balance = data['Balance']
#Convert to only showing year and months
data = data.set_index('Time',inplace=False,drop=True)
data.index = pd.to_datetime(data.index, format='%Y.%m.%d %H:%M:%S')
data.index = data.index.strftime('%Y-%m')

# Compute the initial account balance before any trades
initial_balance = account_balance.iloc[0] - profit_loss.iloc[0]

# Calculate the account balance before each trade
account_balance_before_trade = account_balance.shift(1)
account_balance_before_trade.iloc[0] = initial_balance
# Compute the percentage change made to the account balance for each trade
percentage_change = profit_loss / account_balance_before_trade
# Fill any NaN values that might have occurred
percentage_change.fillna(0, inplace=True)
percentage_change.index=data.index
data['return'] = percentage_change
print(data)

# Save the processed data to a new CSV file with index
output_csv_path = 'A1-DBG-SP.csv'
data.to_csv(output_csv_path)
print(f"Processed data saved to {output_csv_path}")


/opt/anaconda3/envs/quant/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


         Profit   Balance    return
Time                               
2019-03  102.63  10102.63  0.010263
2019-03  -38.30  10064.33 -0.003791
2019-03  137.25  10201.58  0.013637
2019-03    9.91  10211.49  0.000971
2019-03  -24.89  10186.60 -0.002437
...         ...       ...       ...
2024-10 -146.75  42004.93 -0.003481
2024-10    0.00  42004.93  0.000000
2024-10 -241.50  41925.13 -0.005749
2024-10  -31.44  41893.69 -0.000750
2024-10  -70.61  41823.08 -0.001685

[1396 rows x 3 columns]
Processed data saved to A1-DBG-SP.csv


In [ ]:
import pandas as pd
import numpy as np
import glob

def main():
    # Get list of strategy CSV files
    strategy_files = glob.glob('A1*.csv')
    returns_list = []
    strategy_names = []
    
    # Read returns for each strategy
    for file in strategy_files:
        try:
            data = pd.read_csv(file, index_col='Time')
            # Ensure 'Time' is parsed correctly as datetime
            data.index = pd.to_datetime(data.index, errors='coerce')
            
            # Drop rows where 'Time' or 'return' is invalid
            data.dropna(subset=['return'], inplace=True)
            
            # Aggregate duplicate time indices by mean (or could use 'sum', but here mean can ignore the trade frequency significance)
            data = data.groupby(data.index).agg({'return': 'mean'})
            
            # Append results
            returns_list.append(data['return'])
            strategy_names.append(file)
        except Exception as e:
            print(f"Error processing {file}: {e}")
            continue
    
    # Check if any data was successfully loaded
    if not returns_list:
        print("No valid data found in files.")
        return
    
    # Combine returns into a single DataFrame, aligning by date
    returns_df = pd.concat(returns_list, axis=1, keys=strategy_names)
    
    # Uncomment the below line if u wanna drop rows with missing values across strategies
    #returns_df.dropna(inplace=True)
    #Uncomment the below line if u wanna just fill unaligned rows with 0( I think this is best for backtest that may have years differences)
    returns_df.fillna(0, inplace=True)
    
    # Calculate expected returns (mean returns)
    expected_returns = returns_df.mean()
    
    # Calculate the covariance matrix of returns
    cov_matrix = returns_df.cov()
    
    # Compute the inverse of the covariance matrix
    try:
        inv_cov_matrix = np.linalg.inv(cov_matrix.values)
    except np.linalg.LinAlgError:
        # Use pseudo-inverse if covariance matrix is singular
        inv_cov_matrix = np.linalg.pinv(cov_matrix.values)
    
    # Calculate Kelly optimal fractions
    kelly_fractions = inv_cov_matrix @ expected_returns.values
    kelly_fractions = kelly_fractions / np.sum(kelly_fractions)

    
    # Organize results into a DataFrame
    kelly_df = pd.DataFrame({
        'Strategy': strategy_names,
        'Kelly Fraction': kelly_fractions
    })
    
    # Display the optimal allocations
    print("Optimal Kelly Fractions for Each Strategy:")
    print(kelly_df)

if __name__ == "__main__":
    main()



Optimal Kelly Fractions for Each Strategy:
         Strategy  Kelly Fraction
0  A1-DBG-XAU.csv        0.211095
1   A1-DBG-SP.csv        0.682924
2   A1-DBG-EU.csv        0.105981
